<div style="text-align: center; border-bottom: 2px solid #eee; padding-bottom: 15px; margin-bottom: 15px;">
  <p><strong>Ingénierie des Connaissances</strong></p>
  
  <h3 style="margin-top: 10px;">Atelier : Prédiction floue du risque de panne d’une machine industrielle</h3>
</div>

<h4>Objectif:</h4>
<p>
  Concevoir un système intelligent combinant logique floue et Machine Learning pour estimer
  le risque de panne d’une machine à partir de plusieurs indicateurs incertains (température,
  vibrations, âge).
</p>

<h4>Vous apprendrez à :</h4>
<ul>
  <li>Construire un système d’inférence floue complet</li>
  <li>Générer un jeu de données synthétique à partir du modèle flou</li>
  <li>Entraîner un modèle ML pour généraliser la prédiction</li>
  <li>Comparer les deux approches</li>
</ul>

In [1]:
# importation des bibliotheques necessaires

import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

<div style=" border-left: 5px solid #007bff; padding: 10px 15px; margin-bottom: 20px;">
  <h3>Modélisation floue</h3>
  <h4>1. Créez un contrôleur flou avec les variables suivantes :</h4>
</div>

<h5 style="margin-top: 15px;">• Entrées (Antécédents)</h5>

<ul>
  <li>
    <strong>Température</strong> (Univers: <code>[0, 100]</code>)
    <ul>
      <li>Basse: <code>[0, 0, 40]</code></li>
      <li>Normale: <code>[30, 50, 70]</code></li>
      <li>Elevée: <code>[60, 100, 100]</code></li>
    </ul>
  </li>
  <li style="margin-top: 10px;">
    <strong>Vibration</strong> (Univers: <code>[0, 10]</code>)
    <ul>
      <li>Faible: <code>[0, 0, 4]</code></li>
      <li>Moyenne: <code>[2, 5, 8]</code></li>
      <li>Forte: <code>[6, 10, 10]</code></li>
    </ul>
  </li>
  <li style="margin-top: 10px;">
    <strong>Âge</strong> (Univers: <code>[0, 20]</code>)
    <ul>
      <li>Neuf: <code>[0, 0, 7]</code></li>
      <li>Moyen: <code>[5, 10, 15]</code></li>
      <li>Ancien: <code>[12, 20, 20]</code></li>
    </ul>
  </li>
</ul>

<h5 style="margin-top: 15px;">• Sortie (Conséquent)</h5>

<ul>
  <li>
    <strong>Risque de panne</strong> (Univers: <code>[0, 10]</code>)
    <ul>
      <li>Faible: <code>[0, 0, 4]</code></li>
      <li>Moyen: <code>[2, 5, 8]</code></li>
      <li>Elevé: <code>[6, 10, 10]</code></li>
    </ul>
  </li>
</ul>

In [2]:
# definissant les variables d'entree 

temp = ctrl.Antecedent(np.arange(0, 101, 1), 'Température')
vibr = ctrl.Antecedent(np.arange(0, 11, 1), 'Vibration')
age = ctrl.Antecedent(np.arange(0, 21, 1), 'Âge')

# Definissant la variable de la sortie 

risque = ctrl.Consequent(np.arange(0, 11, 1), 'Risque de panne')

In [3]:
# definissant les fonctions d'appartenance

# Fonctions d'appartenance pour Température
temp['Basse'] = fuzz.trimf(temp.universe, [0, 0, 40])
temp['Normale'] = fuzz.trimf(temp.universe, [30, 50, 70])
temp['Elevée'] = fuzz.trimf(temp.universe, [60, 100, 100])

# Fonctions d'appartenance pour Vibration
vibr['Faible'] = fuzz.trimf(vibr.universe, [0, 0, 4])
vibr['Moyenne'] = fuzz.trimf(vibr.universe, [2, 5, 8])
vibr['Forte'] = fuzz.trimf(vibr.universe, [6, 10, 10])

# Fonctions d'appartenance pour Âge
age['Neuf'] = fuzz.trimf(age.universe, [0, 0, 7])
age['Moyen'] = fuzz.trimf(age.universe, [5, 10, 15])
age['Ancien'] = fuzz.trimf(age.universe, [12, 20, 20])

# Fonctions d'appartenance pour Risque
risque['Faible'] = fuzz.trimf(risque.universe, [0, 0, 4])
risque['Moyen'] = fuzz.trimf(risque.universe, [2, 5, 8])
risque['Elevé'] = fuzz.trimf(risque.universe, [6, 10, 10])

<h4>2. Formulez les règles d’inférence floues</h4>
<p>
  Les règles relient les entrées (température, vibration, âge) à la sortie (risque) et représentent le raisonnement d’un expert en maintenance.
</p>

<div style=" border: 1px solid #ddd; border-radius: 5px; padding: 15px; margin-top: 10px;">
  <ul style="list-style-type: none; padding-left: 0; margin: 0;">
    <li style="margin-bottom: 10px;">
      <strong>Règle 1 :</strong> Si la température est <em>élevée</em> <strong>OU</strong> la vibration est <em>forte</em>, <strong>ALORS</strong> le risque est <em>élevé</em>.
    </li>
    <li style="margin-bottom: 10px;">
      <strong>Règle 2 :</strong> Si la machine est <em>ancienne</em> <strong>ET</strong> la vibration est <em>moyenne</em>, <strong>ALORS</strong> le risque est <em>moyen</em>.
    </li>
    <li style="margin-bottom: 10px;">
      <strong>Règle 3 :</strong> Si la température est <em>basse</em> <strong>ET</strong> la vibration est <em>faible</em> <strong>ET</strong> l’âge est <em>neuf</em>, <strong>ALORS</strong> le risque est <em>faible</em>.
    </li>
    <li>
      <strong>Règle 4 :</strong> Si la température est <em>normale</em> <strong>ET</strong> l’âge est <em>moyen</em>, <strong>ALORS</strong> le risque est <em>moyen</em>.
    </li>
  </ul>
</div>

In [4]:
# Ici on Définir les règles d'inférence en traduisant les règles de l'expert
rule1 = ctrl.Rule(temp['Elevée'] | vibr['Forte'], risque['Elevé'])
rule2 = ctrl.Rule(age['Ancien'] & vibr['Moyenne'], risque['Moyen'])
rule3 = ctrl.Rule(temp['Basse'] & vibr['Faible'] & age['Neuf'], risque['Faible'])
rule4 = ctrl.Rule(temp['Normale'] & age['Moyen'], risque['Moyen'])

<h4>3. Créez et simulez le contrôleur :</h4>

In [5]:
# Création du système de contrôle
risque_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])

# Création de la simulation (c'est ce que nous utiliserons pour calculer)
simulation_risque = ctrl.ControlSystemSimulation(risque_ctrl)

<div style=" border-left: 5px solid #007bff; padding: 10px 15px; margin-top: 20px; margin-bottom: 20px;">
    <h3>Machine Learning</h3>
    <h4>1. Générez 2000 échantillons aléatoires de température, vibration et âge.</h4>
</div>


In [10]:
N_ECHANTILLONS = 2000

# Générer des valeurs aléatoires dans les plages définies
input_temp = np.random.uniform(0, 100, N_ECHANTILLONS) # prend 2000 valeurs entre 0 -> 100 
input_vibr = np.random.uniform(0, 10, N_ECHANTILLONS)  # .... entre 0 -> 10
input_age = np.random.uniform(0, 20, N_ECHANTILLONS)   # .... entre 0 -> 20



In [11]:
df = pd.DataFrame({
    'Température': input_temp,
    'Vibration': input_vibr,
    'Âge': input_age
})

df.head()


,Température,Vibration,Âge
0,97.041954,5.026719,19.869731
1,57.321421,1.726907,19.922882
2,66.478102,5.495463,1.782997
3,54.759159,8.893775,18.988133
4,96.806372,8.337544,7.652869


<h4>2. Calculez pour chacun la valeur défuzzifiée du risque à partir du modèle flou.</h4>

In [14]:
# Stocker les résultats
output_risque_flou = []

# Boucle de simulation
for i in range(N_ECHANTILLONS):
    simulation_risque.input['Température'] = input_temp[i]
    simulation_risque.input['Vibration'] = input_vibr[i]
    simulation_risque.input['Âge'] = input_age[i]
    
    # Exécuter la simulation
    simulation_risque.compute()

    # Récupérer la clé correspondant au "risque" (gestion des accents / variations)
    key_name = None
    for k in simulation_risque.output.keys():
        if 'risque' in k.lower():
            key_name = k
            break

    if key_name is None:
        output_risque_flou.append(None)
    else:
        # Stocker le résultat défuzzifié
        output_risque_flou.append(simulation_risque.output[key_name])

<h4>3. Entraînez un modèle de régression pour prédire le risque à partir des trois entrées.</h4>

In [15]:
# Préparer les données pour scikit-learn
X = np.stack([input_temp, input_vibr, input_age], axis=1)
y = np.array(output_risque_flou)

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choisir un modèle de régression (RandomForest est un bon point de départ)
ml_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entraîner le modèle
ml_model.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


<div style="border-left: 5px solid #007bff; padding: 10px 15px; margin-top: 20px; margin-bottom: 20px;">
  <h3>Interprétation et Analyse</h3>
  <h4>1. Comparez les prédictions floues et celles du modèle ML.</h4>
</div>


In [ ]:
# Prédictions du modèle ML sur l'ensemble de test
y_pred_ml = ml_model.predict(X_test)

# Comparer les prédictions ML (y_pred_ml) avec la sortie floue "réelle" (y_test)
print(f"Erreur quadratique moyenne (MSE) : {mean_squared_error(y_test, y_pred_ml)}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_ml, alpha=0.5)
plt.plot([0, 10], [0, 10], 'r--') # Ligne de prédiction parfaite
plt.xlabel("Vraies valeurs (Sortie Floue)")
plt.ylabel("Prédictions (Modèle ML)")
plt.title("Comparaison Prédictions ML vs Sorties Floues")
plt.show()

<h3>2. Analyse des avantages</h3>
<p>Comparaison des deux approches utilisées dans cet atelier :</p>

<h4>Logique Floue</h4>
<ul>
    <li><strong>Transparence :</strong> Les règles sont lisibles et définies par un expert. Vous savez <em>pourquoi</em> le risque est "Élevé".</li>
    <li><strong>Interprétabilité :</strong> Le modèle est facile à expliquer au service de maintenance.</li>
    <li><strong>Pas de données initiales :</strong> Le système peut être construit uniquement sur la base de la connaissance de l'expert, sans nécessiter un jeu de données historique.</li>
</ul>

<h4>Machine Learning</h4>
<ul>
    <li><strong>Généralisation :</strong> Le modèle ML apprend la relation complexe entre les entrées et la sortie. Il approxime la surface de contrôle floue.</li>
    <li><strong>Précision (Rapidité) :</strong> Une fois entraîné, le modèle ML est très rapide pour faire des prédictions (c'est un calcul mathématique direct, pas un processus d'inférence floue).</li>
    <li><strong>Adaptabilité :</strong> Si vous aviez de <em>vraies</em> données de pannes (et non synthétiques), vous pourriez entraîner le ML dessus. Le modèle pourrait alors découvrir des relations que l'expert n'a pas vues et potentiellement devenir plus précis que les règles "humaines".</li>
</ul>
